# PLATON: PLanetary Atmospheric Transmission for Observer Noobs 

**PLATON** is an exoplanetary atmospheric retreival code that fits a sample transmission spectra to observational data from space telescopes. This examples will process the well constrained exoplanet HD2095458b.

An **exoplanet** is a planet that orbits a star other than our own.

A **transit** is the fortuitous event when the planet passes in front of its host star. The depth of a transit is related to the amount of starlight that is absorbed by the exoplanet's atmosphere.  
An **eclipse** is the fortuitous event when the planet passes behind its host star. The depth of an eclipse is related to the amount of planetary thermal energy (IR emission) that is absorbed by the exoplanet's atmosphere above the photosphere (source of IR emission).  

A **transmission spectrum** is a connected sequence of transit depths (area ratio of the planet-to-star size during transit: (Rp/Rs)^2 ) over a range of wavelengths. That is to say that the transit depth (i.e. relative absorption by (planet + atmosphere) / (size of star) ) changes with wavelength because the atmospheric particles (e.g. gas, clouds, haze) have wavelength dependent absoprtion properties -- the atmosphere absorbs more photons at the core of molecular features compared to the wings thereof.

An **emission spectrum** is a connected sequence of eclipse depths (flux ratio of the planet-to-star emission during transit: Fp/Fs ) over a range of wavelengths. That is to say that the eclipse depth (i.e. relative emission by (planet + atmosphere) / (emission by star) ) changes with wavelength because the atmospheric particles (e.g. gas, clouds, haze) have wavelength dependent absoprtion properties -- the atmosphere absorbs more photons at the core of molecular features compared to the wings thereof.

A **Retrieval** is the numerical process (usually *Bayesian*) to invert the transmission or emission spectrum (here transmission) into a measurement of the abundances of particles (gas, clouds, hazes) 

# Model Parameters

### Planet/Star Parameters
* `Rp` - radius of the planet in units of Jupiter Radii
* `Rstar` - radius of the planet in units of Solar Radii
* `Mp` - mass of the planet in units of Jupiter Mass 

### Temperature-Pressure Profile Parameters

The thermal profile used in the model assumes the atmosphere to be in radiative equilibrium based upon the analytic radiative equilibrium temperature profile of Guillot 2010. This profile assumes one downwelling visible channels of radiation and one upwelling stream of thermal emission.

* `Tirr` - temperature at the top of atmosphere in Kelvin
* `logKir` - log of the Planck mean thermal infrared opacity
* `logg1` - log of the ratio of the Planck mean opacities in the visible stream to the thermal stream

### Chemistry Parameters
* `logMet` - log Metallicity
* `logCtoO` - log of the ratio Carbon to Oxygen abundance
* `logPQC` - log of the pressure of Carbon quenching
* `logPQN` - log of the pressure of Nitrogen quenching

### Cloud Parameters
* `logRayAmp` - log of the amplitude of Rayleigh scattering
* `RaySlp` - Slope of Rayleigh scattering
* `logPc` - Pressure level of cloud deck in mbar


---

We are adding the reload functionality to this Jupyter notebook in case the user wishes to make on the fly changes to the included software packages

In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


We are adding the reload functionality to this Jupyter notebook in case the user wishes to make on the fly changes to the included software packages

In [2]:
%matplotlib inline

We are loading the neccesary built in python libraries here

In [3]:
from __future__ import print_function

from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

from multiprocessing import Pool, cpu_count

import os

import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate

from datetime import datetime
from pandas import DataFrame
from sklearn.externals import joblib

Let's set the figure dimensions to nominal values

In [4]:
plt.rcParams['figure.dpi'] = 150
plt.rcParams['figure.figsize'] = (5,5)

If you already have `emcee` installed, then this will load it.  
If you do not already have `emcee` installed, then this install it first, then load it.  

In [5]:
try:
    import emcee
except:
    !pip install git+https://github.com/dfm/emcee
    import emcee

If you already have `nestle` installed, then this will load it.  
If you do not already have `nestle` installed, then this install it first, then load it.  

In [6]:
try:
    import nestle
except:
    !pip install nestle
    import nestle

If you already have `platon` installed, then this will load it.  
If you do not already have `platon` installed, then this install it first, then load it.  

We are using my github fork because it includes modifications for plotting and usage of the progress bar

In [7]:
try:
    import platon
    del platon
except:
    !pip install git+https://github.com/exowanderer/platon

If you already have `pygtc` installed, then this will load it.  
If you do not already have `pygtc` installed, then this install it first, then load it.  

We are using my github fork because it includes modifications for plotting with mulitnest.

In [8]:
try:
    from pygtc import plotGTC
except:
    !pip install git+https://github.com/exowanderer/pygtc
    from pygtc import plotGTC

If you already have `exomast_api` installed, then this will load it.  
If you do not already have `exomast_api` installed, then this will install it first, then load it.

We are using the github repo because it is not on PYPI or Conda yet.

In [9]:
try:
    from exomast_api import exoMAST_API
except:
    !pip install git+https://github.com/exowanderer/exomast_api
    from exomast_api import exoMAST_API

Here we are loading the necessary `platon` library functions, such as `platon.fit_info`, `platon.transit_depth_calculator`, etc

In [10]:
from platon.fit_info import FitInfo
from platon.retriever import Retriever
from platon.constants import R_sun, R_jup, M_jup
from platon.constants import METRES_TO_UM
from platon.transit_depth_calculator import TransitDepthCalculator
from platon.errors import AtmosphereError

# Plotting and Helper Functions

Now we will define useful computation and plotting functions. These are not necessary for `platon` to operate; but they streamline the plotting procedures for later.

Starting with a model that computes the high resolution atmospheric model from the best-fit (or other) parameters derived below.

In [11]:
def compute_new_model(result, fit_info, bayesian_model='emcee', wave_min=0.2e-6, wave_max=9.0e-6, n_theory_pts=500, include_condensation=True):

    wavelengths_theory = np.linspace(wave_min, wave_max, n_theory_pts)
    half_diff_lam = 0.5*np.median(np.diff(wavelengths_theory))

    # Setup calculator to use the theoretical wavelengths
    calculator = TransitDepthCalculator(include_condensation=include_condensation)
    calculator.change_wavelength_bins(np.transpose([wavelengths_theory-half_diff_lam, wavelengths_theory+half_diff_lam]))

    retriever._validate_params(fit_info, calculator)

    # Allocate the best-fit parameters from the `result` class
    if bayesian_model == 'multinest':
        best_params_arr = result.samples[np.argmax(result.logl)]
    elif bayesian_model == 'emcee':
        best_params_arr = result.flatchain[np.argmax(result.flatlnprobability)]
    else:
        raise ValueError("Options for `bayesian_model` (-bm, --bayesianmodel) must be either 'multinest' or 'emcee'")

    best_params_dict = {key:val for key,val in zip(fit_info.fit_param_names, best_params_arr)}

    # Set the static parameteres to the default values
    for key in fit_info.all_params.keys():
        if key not in best_params_dict.keys():
            best_params_dict[key] = fit_info.all_params[key].best_guess

    # Assign the best fit model parameters to necessary variables
    Rs = best_params_dict['Rs']
    Mp = best_params_dict['Mp']
    Rp = best_params_dict['Rp']
    T_eq = best_params_dict['T']
    logZ = best_params_dict['logZ']
    CO_ratio = best_params_dict['CO_ratio']
    log_cloudtop_P = best_params_dict['log_cloudtop_P']
    log_scatt_factor = best_params_dict['log_scatt_factor']
    scatt_slope = best_params_dict['scatt_slope']
    error_multiple = best_params_dict['error_multiple']
    T_star = best_params_dict['T_star']

    T_spot = best_params_dict['T_spot']
    spot_cov_frac = best_params_dict['spot_cov_frac']
    frac_scale_height = best_params_dict['frac_scale_height']
    log_number_density = best_params_dict['log_number_density']
    log_part_size = best_params_dict['log_part_size']
    part_size_std = best_params_dict['part_size_std']
    ri = best_params_dict['ri']

    # Compute best-fit theoretical model
    try:
        wavelengths, calculated_depths = calculator.compute_depths(
            Rs, Mp, Rp, T_eq, logZ, CO_ratio,
            scattering_factor=10**log_scatt_factor, scattering_slope=scatt_slope,
            cloudtop_pressure=10**log_cloudtop_P, T_star=T_star)
        
        return wavelengths, calculated_depths
    except AtmosphereError as error_message:
        return str(error_message)

This next funciton computes the high resolution atmospheric model (see above) and plots the model with the given data.

In [12]:
def plot_high_res_model_and_data(wave_data, t_depths, td_errors, result, fit_info, bayesian_model='emcee', wave_min=0.2e-6, wave_max=9.0e-6, n_theory_pts=500, include_condensation=True):
    # Plot the data on top of the best fit high-resolution model
    
    wave_min = wave_data.min()
    wave_max = wave_data.max()
    
    computed_model = compute_new_model(result=result, fit_info=fit_info, bayesian_model=bayesian_model, 
                                                       wave_min=wave_min, wave_max=wave_max, n_theory_pts=n_theory_pts, 
                                                       include_condensation=include_condensation)

    if isinstance(computed_model,str):
        raise AtmosphereError(computed_model)
    else:
        wavelengths, calculated_depths = computed_model
    
    plt.figure(figsize=(10,5))
    plt.errorbar(METRES_TO_UM * wave_data, t_depths, yerr=td_errors, fmt='.', color='k', zorder=100)
    plt.plot(METRES_TO_UM * wavelengths, calculated_depths)

    plt.xlabel("Wavelength (um)")
    plt.ylabel("Transit depth")
    plt.xscale('log')

    plt.tight_layout()

    if bayesian_model == 'multinest':
        plt.savefig('multinest_best_fit_{}.png'.format(time_stamp))
    elif bayesian_model == 'emcee':
        plt.savefig('emcee_best_fit_{}walkers_{}steps_{}.png'.format(nwalkers, nsteps, time_stamp))
    else:
        raise ValueError("Options for `bayesian_model` (-bm, --bayesianmodel) must be either 'multinest' or 'emcee'")

The next function takes as input the output of either `emcee` or `nestle` and plots the marginalized distribution and correlation plots for the Bayesian analysis below.

In [13]:
def plot_gtc_platon(result, fit_info, time_stamp='NotGiven', 
                    bayesian_model='emcee', fit_param_names=None, 
                    nContourLevels=3, label_fontsize=10, 
                    tick_fontsize=7, figureSize=12):
    
    # GTC: Grand Triangle of Confusion; i.e. Prettier Corner Plot
    if bayesian_model == 'multinest':
        res_flatchain_df = DataFrame(result.samples.copy(), columns=fit_info.fit_param_names)
    elif bayesian_model == 'emcee':
        res_flatchain_df = DataFrame(result.flatchain.copy(), columns=fit_info.fit_param_names)
    else:
        raise ValueError("Options for `bayesian_model` (-bm, --bayesianmodel) must be either 'multinest' or 'emcee'")
    
    if fit_param_names is None: fit_param_names = fit_info.fit_param_names
    
    if 'Rs' in fit_info.fit_param_names: res_flatchain_df['Rs'] = res_flatchain_df['Rs'] / R_sun
    if 'Rp' in fit_info.fit_param_names: res_flatchain_df['Rp'] = res_flatchain_df['Rp'] / R_jup
    if 'Mp' in fit_info.fit_param_names: res_flatchain_df['Mp'] = res_flatchain_df['Mp'] / M_jup
    
    pygtc_out = plotGTC(res_flatchain_df.values,
                        weights=result.weights if bayesian_model == 'multimodel' else None,
                        nContourLevels=nContourLevels, 
                        paramNames=fit_param_names,
                        customLabelFont={'size':label_fontsize},
                        customTickFont={'size':tick_fontsize},
                        figureSize=figureSize);

    if bayesian_model == 'multinest':
        plt.savefig('multinest_gtc_{}.png'.format(time_stamp))
    elif bayesian_model == 'emcee':
        plt.savefig('emcee_gtc_{}walkers_{}steps_{}.png'.format(nwalkers, nsteps, time_stamp))
    else:
        raise ValueError("Options for `bayesian_model` (-bm, --bayesianmodel) must be either 'multinest' or 'emcee'")

These helper functions simply store the measured transit depths of HD209458b at multiple wavelengths, by instrument.

---
# Data processing and Retrieval Pipeline

Here we define the input data for the retrieval algorithm using the `exomast_API` routines to access `exo.mast.stsci.edu`

In [16]:
planet_name = 'HD 209458 b'
hd209458b = exoMAST_API(planet_name)
hd209458b.get_spectra()
wave_bins, wave_errs, depths, errors = hd209458b.planetary_spectra_table.values.T

HTTPError: https://exo.mast.stsci.edu/api/v0.1/exoplanets/identifiers/?name=HD 209458 b generated the error:
<html><title>500: Internal Server Error</title><body>500: Internal Server Error</body></html>

This is the base class for the `platon` retrieval algorithm. It contains all of the necessary functionality below.

In [ ]:
#create a Retriever object
retriever = Retriever()

Here is our initial guess for each of the fitted and non-fitted parameters that `platon` depends on.

In [ ]:
Rs = hd209458b.Rs * R_sun
Mp = hd209458b.Ms * M_jup
Rp = hd209458b.Rp * R_jup
T_eq = hd209458b.Tp
T_star = hd209458b.Teff # Stellar Temperature

logZ = 0
CO_ratio = 0.53 # Solar
log_cloudtop_P = 4 # deep
log_scatt_factor = 0 # non existant
scatt_slope = 4 # Rayleigh scattering
error_multiple = 1 # helps with convergence

Now we will wrapp all of the above parameters into the `platon` specific `fit_info` instance.  
This is a method to contain all and only the necessary information to pass to `platon`.

In [ ]:
#create a FitInfo object and set best guess parameters
fit_info = retriever.get_default_fit_info(
                        Rs=Rs, Mp=Mp, Rp=Rp, T=T_eq,
                        logZ=logZ, CO_ratio=CO_ratio, 
                        log_cloudtop_P=log_cloudtop_P,
                        log_scatt_factor=log_scatt_factor, 
                        scatt_slope=scatt_slope, 
                        error_multiple=error_multiple, 
                        T_star=T_star)

Here we are adding our Bayesian priors onto the `platon` model format.  
Only the parameters listed here will be varrid during the subsequence `emcee` or `nestle` Bayesian estimation routines.

In [ ]:
#Add fitting parameters - this specifies which parameters you want to fit
#e.g. since we have not included cloudtop_P, it will be fixed at the value specified in the constructor
fit_info.add_gaussian_fit_param('Rs', 0.02*R_sun)
fit_info.add_gaussian_fit_param('Mp', 0.04*M_jup)

fit_info.add_uniform_fit_param('Rp', 0.9*R_guess, 1.1*R_guess)
fit_info.add_uniform_fit_param('T', 0.5*T_guess, 1.5*T_guess)
fit_info.add_uniform_fit_param("log_scatt_factor", 0, 1)
fit_info.add_uniform_fit_param("logZ", -1, 3)
fit_info.add_uniform_fit_param("log_cloudtop_P", -0.99, 5)
fit_info.add_uniform_fit_param("error_multiple", 0.5, 5)

This is a time stamp label to associate all outputs from the same run through in the storage device (i.e. hard drive).  
All outputs from this operation will have the same time stamp. We should avoid over writing new results with the same time stamp.

In [ ]:
time_stamp = datetime.utcnow().strftime("%Y%m%d%H%M%S")

These values for the `emcee` are a nominal starting position that will allow us to confirm that the code without programming fails. The results should not be taken as accurate or converged. To perform a robust analysis, we would need ~$10^6$ operations: i.e. nwalkers = 1000; nsteps = 1000. A full, converged chain may take 24+ hours to compute.

Moroever, `emcee` is likely not the best approach to fully understand the atmospheric parameter space.  
It is highly recommended to use `bayesian_model = 'multinest'`, which could take *3+ days* to converge; but is far more likely to accurately measure the parameter space.

This setup will take ~1 minute to run; but the results are not converged, to say the least.

In [ ]:
bayesian_model = 'emcee' # 'multinest'
nwalkers = 50
nsteps = 10

Here is the powerhouse of the `platon` algorithm that operates either the `emcee` or `multinest` Bayesian models. 

We will use the emcee model because it is easier to control how long a completed (not converged) operation will take. Multinest (via `nestle`) is far more likely to accurate probe the full parameter space volume; but it could take ~3 days to converge. Emcee will most likely focus on one mode of many; but we can control how long our computers operate by setting the `nwalkers` and `nsteps` parameters above. 

The nominal use case has shown that each walker and step should take 0.1 seconds.

That is to say that 50 walkers and 20 steps should take 0.1 x 50 x 20 = 100 seconds; and 1000 walkers and 1000 steps should take 0.1 x 1000 x 1000 = 100k seconds = ~28 hours

In [ ]:
if bayesian_model == 'multinest':
    #Use Nested Sampling to do the fitting
    # with ThreadPoolExecutor() as executor:
    # with ProcessPoolExecutor() as executor:
    with Pool(cpu_count()) as executor:
        result = retriever.run_multinest(wave_bins, depths, errors, fit_info, nestle_kwargs={'pool':executor})#, 'bootstrap':0 # bootstrap for `dynesty`
    
    result_dict = {'samples':result.samples, 'weights':result.weights, 'logl':result.logl}
    joblib.dump(result_dict, 'multinest_results_{}.joblib.save'.format(time_stamp))
elif bayesian_model == 'emcee':
    #Use Affine Invariant Ensemble MCMC  to do the fitting
    result = retriever.run_emcee(wave_bins, depths, errors, fit_info, nwalkers=nwalkers, nsteps=nsteps)
    
    result_dict = {'flatchain':result.flatchain, 'flatlnprob':result.flatlnprobability, 'chain':result.chain, 'lnprob':result.lnprobability}
    joblib.dump(result_dict, 'emcee_results_{}walkers_{}steps_{}.joblib.save'.format(nwalkers, nsteps, time_stamp))
else:
    raise ValueError("Options for `bayesian_model` (-bm, --bayesianmodel) must be either 'multinest' or 'emcee'")

---
Next we will plot the best fit model to confirm that our fitting algorithm did not run off a cliff into an unphysical parameter space.  
As long as the blue model is close to the black dots, then we should be happy.  

To attain better convergence, we could modify the atmospheric model *or* iterate the Bayesian longer -- possibly with different priors or intial conditions.

In [ ]:
# Establish the Range in Wavelength to plot high resolution figures
wave_min = wave_bins.min()
wave_max = wave_bins.max()
print('{:.3f} - {:.1f} {}'.format(wave_min*1e6,wave_max*1e6, 'microns'))

wave_data = np.mean(wave_bins, axis=1)

plot_high_res_model_and_data(wave_data=wave_data, t_depths=depths, td_errors=errors, result=result, fit_info=fit_info)

Finally we will examine the distribution and correlations for each of our 8 parameters. The diagrams can be interpretted as:

(1) The 1D histograms represent the 3-sigma width of the marginalized Bayesian Credible Regions: Bayesian version of *confidence interval*.

(2) The 2D histograms represent the $N(N-1)/2$ combinations of correlations between each of our N parameters (N=8 here).  
If the 2D histograms are reasonably circular, then we can assume that the correlations are Gaussian, that is to say that they reflect the intrinsic uncertainty of the measurements; as oppose to correlations between any 2 of the parameters.  
If the 2D histograms show obvious structure (linear trensds, multimodel populations, etc), then we should assume that parameter correlations may have an effect on the reported values and uncertainties. 

If we a linear trend between two physically correlated values (i.e. Rp and Mp), then the explanation is trivial.  
If find a linear trend between two parameters without any known correlations, then we should investigate more closely.  
If we find a non-linear trend between any two parameters, it should be investigated more closely.  
If we see a bimodel or multimodel distribution, then we should investigate further -- nominally, the solution may require more data, which would probably require more observations.

In [ ]:
fit_param_names = ['Rs [Rsun]', 'Mp [Mjup]', 'Rp [Rjup]', 'T [K]', 'log(haze)', 'logZ [FeH]', 'log(P_c) [mbar]', 'err_mod']
plot_gtc_platon(result, fit_info, bayesian_model='emcee', fit_param_names=fit_param_names, nContourLevels=2)

---
# Show Stored Best Fit Solution with Multinest

![title](multinest_best_fit_20181113065432.png)

# Show Stored Grand Triangle of Confusion Solution with Multinest

![title](multinest_gtc_20181113065432.png)